In [17]:
import open3d as o3d
import numpy as np
import copy
import math

def create_camera_transformation_matrix(position, rotation, use_custom_cal=False):
    heading, pitch, roll = rotation
    # Compute rotation matrix
    if use_custom_cal:
        pass
        # rotation_matrix = euler_to_rotation_matrix(*rotation)
    else:
        rotation_matrix = o3d.geometry.get_rotation_matrix_from_zxy(
            [math.radians(heading), math.radians(pitch), math.radians(roll)]
        )

    # Create transformation matrix
    transformation = np.eye(4)
    transformation[:3, :3] = rotation_matrix
    transformation[:3, 3] = position

    return transformation

def convert_coordinate_system_pcd(pcd):
    # z 2 to y 1
    # y 1 to x 0
    # x 0 to z 2

    pcd_points = np.asarray(pcd.points)
    pcd_points[:,[0,1,2]] = pcd_points[:,[0,2,1]]
    # pcd.transform(np.array([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]))

    # pcd.transform([[1, 0, 0, 0], [0, 0, -1, 0], [0, -1, 0, 0], [0, 0, 0, 1]])
    pcd.points = o3d.utility.Vector3dVector(pcd_points)

    return pcd

def generate_axes(scale):
    points = [
        [0, 0, 0],
        [scale, 0, 0],
        [0, scale, 0],
        [0, 0, scale],
    ]
    lines = [
        [0, 1],
        [0, 2],
        [0, 3]
    ]
    colors = [
        [1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]
    ]
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

def prune_point_cloud_cube(pcd, max_distance):
    """
    Prune points that are outside a cube centered at origin with side length 2*max_distance
    """
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors) if pcd.has_colors() else None
    
    # Find points within the cube (|x| <= max_distance, |y| <= max_distance, |z| <= max_distance)
    mask = np.logical_and.reduce((
        np.abs(points[:, 0]) <= max_distance,
        np.abs(points[:, 1]) <= max_distance,
        np.abs(points[:, 2]) <= max_distance
    ))
    
    # Create a new point cloud with only the points within the cube
    pruned_pcd = o3d.geometry.PointCloud()
    pruned_pcd.points = o3d.utility.Vector3dVector(points[mask])
    if colors is not None:
        pruned_pcd.colors = o3d.utility.Vector3dVector(colors[mask])
    
    return pruned_pcd

def visualize_boundary_cube(max_distance):
    """
    Create a wireframe cube to visualize the boundary
    """
    points = [
        [-max_distance, -max_distance, -max_distance],  # 0
        [max_distance, -max_distance, -max_distance],   # 1
        [-max_distance, max_distance, -max_distance],   # 2
        [max_distance, max_distance, -max_distance],    # 3
        [-max_distance, -max_distance, max_distance],   # 4
        [max_distance, -max_distance, max_distance],    # 5
        [-max_distance, max_distance, max_distance],    # 6
        [max_distance, max_distance, max_distance]      # 7
    ]
    
    lines = [
        [0, 1], [0, 2], [1, 3], [2, 3],  # bottom face
        [4, 5], [4, 6], [5, 7], [6, 7],  # top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # connecting edges
    ]
    
    colors = [[0.8, 0.8, 0.8] for _ in range(len(lines))]
    
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(points),
        lines=o3d.utility.Vector2iVector(lines)
    )
    
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

# Create camera visualizations
camera = o3d.geometry.LineSet.create_camera_visualization(
    intrinsic=o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
    ),
    extrinsic=np.eye(4),
    scale=10
)

camera2 = o3d.geometry.LineSet.create_camera_visualization(
    intrinsic=o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault
    ),
    extrinsic=np.eye(4),
    scale=10
)

camera_position = (-9.886853564542303, 27.4286920728011, -10.59119235780782)  # (X, Y, Z)
camera_rotation = (41.69092292632279, 58.46206628747401, 3.678327983550952)  # (Omega, Phi, Kappa)

x, y, alt = camera_position
heading, pitch, roll = camera_rotation

# Create transformation matrix
matrix = create_camera_transformation_matrix([x, y, alt], [heading, pitch, roll])

rotation = o3d.geometry.get_rotation_matrix_from_zxy([math.radians(heading), math.radians(pitch), math.radians(roll)])

# Apply the same transformation to both cameras
camera = camera.rotate(rotation, center=[0, 0, 0])
camera.translate([x, y, alt])
camera2 = camera2.transform(matrix)  # Use the same matrix for both cameras

# Load point cloud
ply_file = "C:/Users/nguye/Desktop/3d/test_data/test.ply"
pcd = o3d.io.read_point_cloud(ply_file)

# Convert coordinate system
pcd = convert_coordinate_system_pcd(pcd)

# Set the maximum distance for pruning (adjust as needed)
max_distance = 50.0

# Prune the point cloud
pruned_pcd = prune_point_cloud_cube(pcd, max_distance)

# Create boundary visualization
boundary_cube = visualize_boundary_cube(max_distance)

# Visualize the point cloud with cameras and boundary
o3d.visualization.draw_geometries([
    pruned_pcd, 
    generate_axes(100), 
    camera, 
    camera2, 
    boundary_cube
])